In [2]:
import cv2
import numpy as np

# Đường kẻ xác định hướng xe
LINE_POSITION = 500

# Kích thước của làn đường
LANE_LEFT_AREA = (0, 400, 500, 720)  # (x1, y1, x2, y2)
LANE_RIGHT_AREA = (650, 400, 1200, 720)  # (x1, y1, x2, y2)

# Phát hiện xe sử dụng Thresholding, Canny Edge Detection và xử lý morphology
def detect_vehicles_with_threshold(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Thêm bước Threshold
    _, thresholded = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    
    # Làm mờ để giảm nhiễu
    blurred = cv2.GaussianBlur(thresholded, (3, 3), 0)
    
    # Phát hiện cạnh Canny
    edges = cv2.Canny(blurred, 150, 200)
    
    # Phép giãn nở để làm dày các đường biên
    kernel = np.ones((3, 3), np.uint8)
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)
   
    # Phép ăn mòn để loại bỏ nhiễu
    edges_eroded = cv2.erode(edges_dilated, kernel, iterations=1)
    
    # Tìm các contour và hierarchies
    contours, hierarchy = cv2.findContours(edges_eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Lọc để chỉ giữ lại các parent contours
    parent_contours = [contour for i, contour in enumerate(contours) if hierarchy[0][i][3] == -1]
    return parent_contours

# Kiểm tra xe có nằm trong vùng làn đường không
def is_vehicle_in_lane(x, y, w, h, lane_area):
    return (lane_area[0] <= x + w//2 <= lane_area[2]) and (lane_area[1] <= y + h//2 <= lane_area[3])

# Khởi tạo video và các đối tượng OpenCV
cap = cv2.VideoCapture(r'video.mp4')
vehicle_directions = {}  # Từ điển lưu hướng di chuyển của xe

# Kiểm tra nếu video có mở thành công
if not cap.isOpened():
    print("Cannot open video file.")
    exit()

# Lấy thông số về kích thước video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Khởi tạo đối tượng VideoWriter để lưu video output
output = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    contours = detect_vehicles_with_threshold(frame)
    
    left_lane_count = 0  # Reset at each frame
    right_lane_count = 0  # Reset at each frame
    detected_boxes = []
    vehicle_positions = {}
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)

        # Kiểm tra diện tích để loại bỏ nhiễu
        if 300 < area and h > 30 and 1.5 < w/h:
            detected_boxes.append((x, y, w, h))

    for box in detected_boxes:
        x, y, w, h = box
        
        # Vị trí trung tâm của bounding box
        center_y = y + h // 2  
        center_x = x + w // 2

        # Kiểm tra xe trong làn đường bên trái
        if is_vehicle_in_lane(x, y, w, h, LANE_LEFT_AREA):
            left_lane_count += 1

        # Kiểm tra xe trong làn đường bên phải
        elif is_vehicle_in_lane(x, y, w, h, LANE_RIGHT_AREA):
            right_lane_count += 1

        # Determine color based on direction
        if center_y < LINE_POSITION:
            color = (0, 0, 255)  # Red for downward
        else:
            color = (0, 255, 0)  # Blue for upward

        # Cập nhật vị trí mới cho xe trong từ điển
        vehicle_directions[box] = center_y
        
        # Vẽ bounding box với màu tương ứng
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Vẽ vùng diện tích cho các làn đường
    cv2.rectangle(frame, (LANE_LEFT_AREA[0], LANE_LEFT_AREA[1]), (LANE_LEFT_AREA[2], LANE_LEFT_AREA[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (LANE_RIGHT_AREA[0], LANE_RIGHT_AREA[1]), (LANE_RIGHT_AREA[2], LANE_RIGHT_AREA[3]), (255, 0, 0), 2)
    cv2.line(frame, (0, LINE_POSITION), (frame.shape[1], LINE_POSITION), (255, 255, 255), 2)

    # Hiển thị đếm số lượng xe trong mỗi làn
    cv2.putText(frame, f'Làn trái: {left_lane_count} | Làn phải: {right_lane_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Ghi frame đã xử lý vào video output
    output.write(frame)

    # Hiển thị frame
    cv2.imshow('Vehicle Detection', frame)

    if cv2.waitKey(30) & 0xFF == 27:  # Nhấn Esc để thoát
        break

# Giải phóng tài nguyên
cap.release()
output.release()
cv2.destroyAllWindows()
